## Imports and library

In [69]:
# import libraries
import hashlib
import json
import binascii

import datetime
import collections

In [70]:
!pip install Crypto

  Using cached crypto-1.4.1-py2.py3-none-any.whl.metadata (3.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 kB 12.8 MB/s eta 0:00:0000:01


In [71]:
# following imports are required by PKI
import Cryptodome
import Cryptodome.Random
from Cryptodome.Hash import SHA
from Cryptodome.PublicKey import RSA
from Cryptodome.Signature import PKCS1_v1_5

## Python Blockchain - Client Class
The Client class generates the private and public keys by using the built-in Python RSA algorithm. The interested reader may refer to this tutorial for the implementation of RSA. During the object initialization, we create private and public keys and store their values in the instance variable.
Note that you should never lose your private key. For record keeping, the generated private key may be copied on a secured external storage or you may simply write down the ASCII representation of it on a piece of paper.

The generated public key will be used as the clients identity. For this, we define a property called identity that returns the HEX representation of the public key.

The identity is unique to each client and can be made publicly available. Anybody would be able to send virtual currency to you using this identity and it will get added to your wallet.

The full code for the Client class is shown here −

In [72]:
class Client:
   def __init__(self):
      random = Cryptodome.Random.new().read
      self._private_key = RSA.generate(1024, random)
      self._public_key = self._private_key.publickey()
      self._signer = PKCS1_v1_5.new(self._private_key)

   @property
   def identity(self):
      return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')

In [74]:
Dinesh = Client()
print(f"Hash of {Dinesh} : \n{Dinesh.identity}")

Hash of <__main__.Client object at 0x7c9b72f6c2f0> : 
30819f300d06092a864886f70d010101050003818d0030818902818100bf0cccdb3fc601a627c33c455171454702164ebd6d9334a5404b07c48dcf289600413b300092ead12f0a8880cd7f22988365e7b30d720eddbb79274e425b5d313d7638fcfe5bc67c21a2f388460321e9e5cfbb3ef96e3f57dfdb6eebfbe330dd88fdb074116520f7cb3deb2e48832de9f7c06ea83cb9e7a48a177110ef1a7ee50203010001


## Python Blockchain - Transaction Class
In this chapter, let us create a Transaction class so that a client will be able to send money to somebody. Note that a client can be both a sender or a recipient of the money. When you want to receive money, some other sender will create a transaction and specify your public address in it. We define the initialization of a transaction class as follows −

The init method takes three parameters − the senders public key, the recipients public key, and the amount to be sent. These are stored in the instance variables for use by other methods. Additionally, we create one more variable for storing the time of transaction.

Next, we write a utility method called to_dict that combines all the four above-mentioned instance variables in a dictionary object. This is just to put the entire transaction information accessible through a single variable.

As you know from the earlier tutorial that the first block in the blockchain is a Genesis block. The Genesis block contains the first transaction initiated by the creator of the blockchain. The identity of this person may be kept a secret like in the case of Bitcoins. So when this first transaction is created, the creator may just send his identity as Genesis. Thus, while creating the dictionary, we check if the sender is Genesis and if so we simply assign some string value to the identity variable; else, we assign the senders identity to the identity variable.

Finally, we will sign this dictionary object using the private key of the sender. As before, we use the built-in PKI with SHA algorithm. The generated signature is decoded to get the ASCII representation for printing and storing it in our blockchain. The sign_transaction method code is shown here 

In [75]:
class Transaction:
    
    def __init__(self, sender, recipient, value):
        self.sender = sender
        self.recipient = recipient
        self.value = value
        self.time = str(datetime.datetime.now())
        
    def to_dict(self):
        if self.sender == "Genesis":
            sender_identity = "Genesis"
        else:
            sender_identity = self.sender.identity
            
        reciever_identity = self.recipient.identity
        transaction_value = self.value
            

        return collections.OrderedDict(
                {
                    'sender': sender_identity,
                    'recipient': reciever_identity,
                    'value': transaction_value,
                    'time' : self.time
                }
            )
    
    def sign_transaction(self):
        private_key = self.sender._private_key
        signer = PKCS1_v1_5.new(private_key)
        h = SHA.new(str(self.to_dict()).encode('utf8'))
        
        return binascii.hexlify(signer.sign(h)).decode('ascii')


In [7]:
Ramesh = Client()
print(f"Hash of {Ramesh} : \n{Ramesh.identity}")

Hash of <__main__.Client object at 0x7c9b73040f50> : 
30819f300d06092a864886f70d010101050003818d0030818902818100b2746b95e8c6c1ebaed72d0c1ed3e0689d9d3cc184781bc69ff3e0b5dde40e31aca316d4956816c43b9ce215574f3f46c2409b2e0394aeec017a714e8389fb65cc00e53811f365097112cc97692bcb6993c4837a52fba1dbcb529fe6e3c71a6f01a502b15ca88a1f94272cc851b6e18c0d3d2b4bb31f920b74a6cb54e084f4850203010001


In [8]:
print(True if Ramesh.identity == Dinesh.identity else False)

False


In [77]:
t = Transaction(
   sender=Dinesh,
   recipient=Ramesh,
   value=5.0
)

In [78]:
dict_1 = t.to_dict()
print(dict_1)

signature = t.sign_transaction()
print(signature)

OrderedDict({'sender': '30819f300d06092a864886f70d010101050003818d0030818902818100bf0cccdb3fc601a627c33c455171454702164ebd6d9334a5404b07c48dcf289600413b300092ead12f0a8880cd7f22988365e7b30d720eddbb79274e425b5d313d7638fcfe5bc67c21a2f388460321e9e5cfbb3ef96e3f57dfdb6eebfbe330dd88fdb074116520f7cb3deb2e48832de9f7c06ea83cb9e7a48a177110ef1a7ee50203010001', 'recipient': '30819f300d06092a864886f70d010101050003818d0030818902818100b2746b95e8c6c1ebaed72d0c1ed3e0689d9d3cc184781bc69ff3e0b5dde40e31aca316d4956816c43b9ce215574f3f46c2409b2e0394aeec017a714e8389fb65cc00e53811f365097112cc97692bcb6993c4837a52fba1dbcb529fe6e3c71a6f01a502b15ca88a1f94272cc851b6e18c0d3d2b4bb31f920b74a6cb54e084f4850203010001', 'value': 5.0, 'time': '2025-11-30 14:18:24.808833'})
a8e5d00cdd2acb498e3fed9e114ee86e9ea8fc92d967c65aca5cfdd1a055df9b8a26435960dc2cc90ecec485f1dc56d6478591ada2fbfb859d612b14602f03cd07ffebb4f24d9a11c7331cf2427306e7dc238acc2a5a25a6f90a184f195bcc3f37a7ac708d2c786435de86943c4193d5369ed6b95ff744ccd3bf35371b126de

## Creating Multiple Transactions
The transactions made by various clients are queued in the system; the miners pick up the transactions from this queue and add it to the block. They will then mine the block and the winning miner would have the privilege of adding the block to the blockchain and thereby earn some money for himself.

We will describe this mining process later when we discuss the creation of the blockchain. Before we write the code for multiple transactions, let us add a small utility function to print the contents of a given transaction.

In [79]:
def get_diff(data):
    s = data["sender"]
    r = data["recipient"]

    s1 = ""
    r1 = ""

    for a, b in zip(s, r):
        if a != b:
            s1 += a
            r1 += b

    return str(s1) , str(r1), str(data['value']), data['time']

In [80]:
def display_trans(transaction):
    dict = transaction.to_dict()
    sender, recipient, value, time = get_diff(dict)
    print (f"sender hash     : {sender}")
    print (f"recipeient hash : {recipient}")
    print (f"value           : {value}")
    print (f"time            : {time}")


In [81]:
transactions = []
signatures = []

Seema = Client()
Vijay = Client()

In [82]:
# Example transaction
t1 = Transaction(
   Dinesh,
   Ramesh,
   15.0
)

transactions.append(t1)
transactions.append(Transaction(Seema, Vijay, 3.0))
transactions.append(Transaction(Ramesh, Vijay, 1.4))
transactions.append(Transaction(Vijay, Dinesh, 2.1))
transactions.append(Transaction(Ramesh, Seema, 1.0))

In [83]:
for transaction in transactions:
    signatures.append(transaction.sign_transaction())

In [84]:
transactions

In [86]:
signatures

['a5b6882da903d24bb4de002a3edf3a169166b774629775606ecef8abf1a5971add7763eb43d3ae8589a62e7e8097bfb4b2d251f5eb2d88b7e59606918002c5f70076ca4a3ed5522ae47d0b3955bd075358b8d4d44af577fa6e4a05182715b97ef4744d98f2b175720ba7088d70b4b11729e7b9d5926c9651bd33f4be7c7dcf22',
 '3a5e39d570f7208166b9908effeaccde83f8213ce041804bd27d4cabcaf8ae7bfc0a4b1fedeec455fc98ddcf71ca3949f1ff5b2eb0614c13d21e9aa82c0fdb45057244c00da34bcf74ea24aa81673898b010b0051ae415d5bb6ff6656a6c1548455b9893c0f4f80f616e33f31a97c3c1e87accc4e82f454d2cb2a25623657cd8',
 '805f289df83e34692727df82b6b71182e55d12fb44e48c4866eafaeb3ddb538f6134fce0f1596338b8303ea192e800b7a6477ad72a81a2c4651aeb5493f00554181396ab1d8245199a29fe7ccb66ea8fca6ca9a0baae01d1cbd68cf3786b8d7da53fd9b75f8f02aa4fb7f478ef620a0c627b1c920efa7f25aa6f841b90941706',
 'c823c089c1ee04d5430a2304147efd223a4d3b5ec321cba82fede3159ed165fc81f4f5b4e03d8efb8da9914087a7d5319c607d368d6570af4cf4c69d865d10f82c931204731588975bee770a7d3e126195e6141c3d625b28a66d569fb47dc24c5fd8f4ed2a5b0fd0875e226

In [27]:
number = 1
for transaction in transactions:
   print(f"Transaction Number and Hash : {number} -> {transaction.sign_transaction()}")
   print()
   display_trans(transaction)
   number += 1
   print("==========================================================")
   print()

Transaction Number and Hash : 1 -> 90ea01e4afbea3b34f176bd3c89629e58cbc55930e784ac48c1dadb60231ab86eed7611112f0e6b2cb7777571c0d27e3bcc0a446e83a3f904b535f41956d2605052d0d04dd5f50814737ab4e9517f0910132826b7165356c9aec390e8206ffd51dfa573b4aa8957ead7ed864bf4907ab6f9ed7d54695e4b3e36afaca4376dc46

sender hash     : a7377442b0e9dcf4ff0bb114f0b261bfa245fb256d45af0d2bea354000f9bd652d77906deb868c694ae7cd979cccedf691ad5b7217e401bdec7561e9bea47bdbc32e4c3d4abb093b9bba8a8ba6ec36e826001d75e503554d071d21b050eaedda8f285b5e56dafdb4fc4c25226e6d42887ed5a72ebc7708c7b82b
recipeient hash : b2746b95e8c6c1ebaed72d0c1ed3e0689d9d3cc184781bc69ff3e0b5dde40e31aca316d4956816c43b9ce2155743f46c2409b2e034aeec017714e8389fb65cc00e53811f365097112cc97692bb6993c4837a52fba1dbcb529fee3c71a6f1a502b15a88a1f9422cc851b6e18c0d3bb31f920b74a6cb54e084f485
value           : 15.0
time            : 2025-11-30 12:00:19.109270

Transaction Number and Hash : 2 -> 90a1780c253e6f685f02be69342efae68c38e731fc6ee966921c86e24487a54281cc7ca6f3d77

## Python Blockchain - Block Class
A block consists of a varying number of transactions. For simplicity, in our case we will assume that the block consists of a fixed number of transactions, which is three in this case. As the block needs to store the list of these three transactions, we will declare an instance variable called verified_transactions as follows −

We have named this variable as verified_transactions to indicate that only the verified valid transactions will be added to the block. Each block also holds the hash value of the previous block, so that the chain of blocks becomes immutable.

To store the previous hash, we declare an instance variable as follows −

Finally, we declare one more variable called Nonce for storing the nonce created by the miner during the mining process.
As each block needs the value of the previous blocks hash we declare a global variable called last_block_hash as follows −

In [31]:
def sha256(message):
    return hashlib.sha256(message.encode()).hexdigest()

In [32]:
class Block:
   def __init__(self, verified_transactions, previous_block_hash, Nonce):
      self.verified_transactions = verified_transactions
      self.previous_block_hash = previous_block_hash
      self.Nonce = Nonce
      
      self.block_data = f"{' - '.join(verified_transactions)} - {previous_block_hash} - {Nonce}"
      self.block_hash = sha256(self.block_data)

In [43]:
def mine(transactions, previous_block_hash, difficulty=2):
    Nonce_limit = 10 ** 10 # implies to 10^10 itearation range
    prefix = '0' * difficulty
    
    for nonce in range(Nonce_limit):
        block = Block(transactions, previous_block_hash, nonce)
        digest = block.block_hash

        if digest.startswith(prefix):
            print(f"Nonce found: {nonce}")
            return block
        
    return "Nonce not found"

In [44]:
block0 = mine(transactions=signatures, previous_block_hash="0000000000000000", difficulty=2)

Nonce found: 709


In [64]:
print(f"Block => block0")
print(f"Block Nonce => {block0.Nonce}")
print()
print("List of Transactions in block0")
i = 1
for transaction in block0.verified_transactions:
    print(f"Transaction {i} Hash => {transaction}")
    i += 1

print()
print(f"Current Block Hash => {block0.block_hash}")
print(f"Previous Block Hash => {block0.previous_block_hash}")

Block => block0
Block Nonce => 709

List of Transactions in block0
Transaction 1 Hash => 90ea01e4afbea3b34f176bd3c89629e58cbc55930e784ac48c1dadb60231ab86eed7611112f0e6b2cb7777571c0d27e3bcc0a446e83a3f904b535f41956d2605052d0d04dd5f50814737ab4e9517f0910132826b7165356c9aec390e8206ffd51dfa573b4aa8957ead7ed864bf4907ab6f9ed7d54695e4b3e36afaca4376dc46
Transaction 2 Hash => 90a1780c253e6f685f02be69342efae68c38e731fc6ee966921c86e24487a54281cc7ca6f3d77af78d6a33513da038cad88e22f869826aaa51e6a9a4ae5cf3b6e73d5651ea133d5354ac492626a1e7d1c34ad9ef34cba6781c9b3dcf2cd2235550622e51f6c7bc2538326972ed155531813b9f1f3c474231b134d056112cccf0
Transaction 3 Hash => 77b6cb144eedea3662f7aee51d47a98eb85bc92eac167342965baf444295694e9075e3dd9800e309f9cc03b5505eb3f38cddfc837bc16c65f1e06f553dc7989c0da370c37f9edc59394433108f0cd728f72442b938e806e20719671d2197575341f217afd613a903a84778ed15803fb100cc795c637a4c769d1fb2eb6e34f1ae
Transaction 4 Hash => b395ef84fbe4ee92944c47ce3d0f46ab050d350b6dfe108a4197ddf20fa45b0b8708a9c6c7

## Implementing Blockchain

clients -> transactions -> blocks

In [76]:
# new clients

John = Client()
Alice = Client()
Bob = Client()


In [ ]:
# Transactions
t6 = Transaction(John, Alice, 5.0)
t7 = Transaction(John, Bob, 2.0)
t8 = Transaction(Bob, Dinesh, 1.2)

t9 = Transaction(Dinesh, John, 1.0)
t10 = Transaction(Alice, John, 2.0)
t11 = Transaction(John, Ramesh, 2.0)
t11 = Transaction(Seema, Vijay, 2.0)
t12 = Transaction(Vijay, Ramesh, 1.0)

In [91]:
new_transactions = [t6, t7, t8]
new_signatures = []
for transaction in new_transactions:
    new_signatures.append(transaction.sign_transaction())


In [92]:
new_transactions


In [93]:
new_signatures

['c5640087153b00883d3fc29098652aebadcc49e3f343c4bd198e9384327b912dc804cffe2f67a15c2935ee79c99012ada66f429b7ab6dc9bc94f9b55cf5434dfb58b076da5f2d7ec6319f79f5f129a52c75b16db68ea154ba9721315e9ca2cbaa2b0c013b2cc812a0b548c3c200f22d5c32b5f9ff72150c1fc07367ce5b8e810',
 '65ce9ab337a79ac522d09577fb0b643c724100dbbbe559804bef8e1b3563feb702bc849497e19302814bc7fde0bb081b2a16be15c9aa11f10a0abb9228f649cd9782ced64638643b5790c2c2d5a9bc7b8b701abd39d6fbde8067cc0507c1e3bcad42c58a80ba4f7061d814e84fa327604a8710e38f53ae88a7a8aba6a6071997',
 '4698b34ec5720c89af7cbfcddc8f503ab7ea437f093b7742168d771b6e747d165456cd98ce357bf18a5ab4e5713c99a3cb532a2ac4abdde01ff84346acec640f39368579167f5934bf39027e7a8db258d69002ddde398d3e68021fb74916b384d20bec180798b55fe2e0110dd1ca22ed17b8f490f2ec8369d10fc2bdfc9ecdc6']

In [94]:
block1 = mine(transactions=new_signatures, previous_block_hash=block0.block_hash, difficulty=2)

Nonce found: 156


In [95]:
last_transactions = [t9, t10, t11, t12]
last_signatures = []
for transaction in last_transactions:
    last_signatures.append(transaction.sign_transaction())


In [96]:
block2 = mine(transactions=last_signatures, previous_block_hash=block1.block_hash, difficulty=2)

Nonce found: 101


In [97]:
# Blockchain of blocks

Blocks = [block0, block1, block2]

for i, block in enumerate(Blocks):
    print(f"Block Number : {i}")
    print(f"Nonce        : {block.Nonce}")
    print(f"Block Hash   : {block.block_hash}")
    print(f"Prev. Hash   : {block.previous_block_hash}")
    print("-----------------------------------------------------")

Block Number : 0
Nonce        : 709
Block Hash   : 005e9c7aa7eda2b879c95dcfe2c91fcf0f8964ca7c46e8eb125e2a738261e391
Prev. Hash   : 0000000000000000
-----------------------------------------------------
Block Number : 1
Nonce        : 156
Block Hash   : 0092e6a2952618b5470610bbbafd512f8be86bfb9e42f114ad9a73b934b33308
Prev. Hash   : 005e9c7aa7eda2b879c95dcfe2c91fcf0f8964ca7c46e8eb125e2a738261e391
-----------------------------------------------------
Block Number : 2
Nonce        : 101
Block Hash   : 001c1c18fc57b23ae99b8e22dbff7d8735fdd215e96f771644878d5cd4182b35
Prev. Hash   : 0092e6a2952618b5470610bbbafd512f8be86bfb9e42f114ad9a73b934b33308
-----------------------------------------------------
